In [13]:
import numpy as np
import matplotlib.pyplot as plt
import iisignature as iis
from gtda.homology import FlagserPersistence
from gtda.homology import VietorisRipsPersistence
import gudhi

In [1]:
def lead_matrix_1(mv_time_series):
    N=mv_time_series.shape[1]
    sig=iis.sig(mv_time_series,2)
    S=sig[N:].reshape(N,N)
    L=(S-S.T)/2
    return L
def lead_matrix_2(mv_time_series):
    N=mv_time_series.shape[1]
    sig=iis.sig(mv_time_series,2)
    S=sig[N:].reshape(N,N)
    L=(S-S.T)/2
    a=np.max(np.abs(L))
    L2=L/a
    return L2    

In [15]:
def lead_tensor_1(mv_time_series):
    N=mv_time_series.shape[1]
    sig=iis.sig(mv_time_series,3)
    l=N+N*N
    S=sig[l:].reshape(N,N,N)
    T=(S-S.transpose((0,2,1))+S.transpose((2,0,1))-S.transpose((2,1,0))+S.transpose((1,2,0))-S.transpose((1,0,2)))/6
    return T

In [16]:
def rev_undirected(L):
    l=L.shape[0]
    a=np.max(np.abs(L))
    ADM=a*np.ones((l,l))-a*np.identity(l)-np.abs(L)
    VR= VietorisRipsPersistence(metric="precomputed")
    dgm= VR.fit_transform([ADM])
    return dgm

In [17]:
def rev_directed(L):
    l=L.shape[0]
    a=np.max(np.abs(L))
    for i in range(l):
        for j in range(l):
            if L[j,i]<0:
                L[j,i]=0
                
    ADM=a*np.ones((l,l))-a*np.identity(l)-L            
    dgm=FlagserPersistence().fit_transform([ADM])
    return dgm

In [18]:
def vertex_edge_tensor_cofl(mv_time_series):
    N=mv_time_series.shape[1]
    l=mv_time_series.shape[0]
    T=np.zeros((N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if ((not (i==j)) and (not (i==k)) and (not (j==k))):
                    store=np.zeros((l,2))
                    store[:,1]=mv_time_series[:,j]*mv_time_series[:,k]
                    store[:,0]=mv_time_series[:,i]
                    sig=iis.sig(store,2)
                    area_cofl=(sig[3]-sig[4])/2
                    T[i,j,k]=area_cofl
                    
    return T
    

In [19]:
def vertex_edge_tensor_conv(mv_time_series):
    N=mv_time_series.shape[1]
    l=mv_time_series.shape[0]
    T=np.zeros((N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if ((not (i==j)) and (not (i==k)) and (not (j==k))):
                    store=np.zeros((l,2))
                    store[:,1]=np.convolve(mv_time_series[:,j],mv_time_series[:,k],"same")
                    store[:,0]=mv_time_series[:,i]
                    sig=iis.sig(store,2)
                    area_conv=(sig[3]-sig[4])/2
                    T[i,j,k]=area_conv
                    
    return T
    

In [20]:
def create_filtration_2(Lead_matrix, Lead_tensor,indifferent_value):
    

    # Creating the list of simplicial complex with all the edges and triangles
    sc=gudhi.SimplexTree()
    list_simplices = []
    N=Lead_matrix.shape[0]
    for i in range(N):
        for j in range(N):
            if np.abs(Lead_matrix[i,j])<indifferent_value:
                Lead_matrix[i,j]=0
            for k in range(N):
                if np.abs(Lead_tensor[i,j,k])<indifferent_value:
                    Lead_tensor[i,j,k]=0

        

        # Selecting the extremal weight between edges and triplets. It will be assigned to all the nodes (i.e. nodes enter at the same instant)
    m_weight = np.max([np.ceil(np.max(np.abs(Lead_matrix))), np.ceil(np.max(np.abs(Lead_tensor)))])
    # Adding all the nodes from the beginning with the same weights
    for i in range(N):
        list_simplices.append(([i], m_weight))

        # Adding the edges:
        # Also, modify the signs of the weights to correct the z-score so that: if the edge signal is fully coherent, then assign a positive sign, otherwise negative
    for i in range(N):
        for j in range(i,N):
            if (not i==j):
                weight=np.abs(Lead_matrix[i,j])
                list_simplices.append(([i,j], weight))

        # Adding the triplets
        # Here I modify the signs of the weights, if it is fully coherent I assign a positive sign, otherwise negative
    for i in range(N):
        for j in range(i,N):
            for k in range(j,N):
                if ((not (i==j)) and (not(j==k)) and (not(i==k))):
                    weight=np.abs(Lead_tensor[i,j,k])
                    list_simplices.append(([i,j,k], weight))

    sorted_simplices = sorted(list_simplices, key=lambda x: x[1], reverse=True)

        # Remove the violations
    list_violating_triangles = []
    set_simplices = set()
    counter = 0
    triangles_count = 0
    violation_triangles = 0
    violation_triangles_negativeterms = 0

        # Loop over the sorted simplices, and flippling the sign of all the weights (so that the points in the persistence diagram are above the diagonal)
    for index, i in enumerate(sorted_simplices):
        simplices, weight = i

            # If the current simplex is an edge or a node, then I will immediately include it
        if len(simplices) <= 2:
            sc.insert(simplices, -weight)
            set_simplices.add(tuple(simplices))
            counter += 1
        else:
        # If the current simplex is a triplet, I check whether all the sub-simplices have been included.
            flag = 0
            n0=simplices[0]
            n1=simplices[1]        
            n2=simplices[2]
            if (n0,n1) in set_simplices:
                flag=flag+1
            if (n1,n2) in set_simplices:
                flag=flag+1
            if (n0,n2) in set_simplices:
                flag=flag+1

                # If all the sub-simplices already belong to the set, then I add it in the filtration
            if flag == 3:
                set_simplices.add(tuple(simplices))
                sc.insert(simplices, -weight)
                counter += 1
                triangles_count += 1
            else:
                violation_triangles += 1
                list_violating_triangles.append((simplices, np.abs(weight), 3 - flag))

# Fraction of positive triangle discarderd (a.k.a. the hyper coherence)
    hyper_coherence = (1.0 * violation_triangles) /(triangles_count + violation_triangles)
        
    return(sc, list_violating_triangles,hyper_coherence)

In [25]:
def create_diagram_directly_2(mv_time_series, indifferent_value):
    L=lead_matrix_1(mv_time_series)
    T=lead_tensor_1(mv_time_series)
    f=create_filtration_2(L,T,indifferent_value)
    sc=f[0]    
    dgm=sc.persistence()
    return dgm

In [29]:
def hyper_coherence(mv_time_series,indifferent_value):
    L=lead_matrix_1(mv_time_series)
    T=lead_tensor_1(mv_time_series)
    f=create_filtration_2(L,T,indifferent_value)
    hyp=f[2]
    return hyp

In [1]:
def create_diagram_directly_3(mv_time_series, indifferent_value):
    L=lead_matrix_1(mv_time_series)
    T=lead_tensor_1(mv_time_series)
    f=create_filtration_2(L,T,indifferent_value)
    sc=f[0]
    hyp=f[2]
    dgm=sc.persistence()
    return dgm,hyp

In [3]:
def entropy_gu_ad(dgm):
    total_0=0
    total_1=0
    l_0=[]
    l_1=[]
    l=len(dgm)
    for i in range(l):
        if dgm[i][0]==1:
            if (not (dgm[i][1][1]==np.inf)):
                length=dgm[i][1][1]-dgm[i][1][0]
                total_1=total_1+length
                l_1.append(length)
        if dgm[i][0]==0:
            if (not (dgm[i][1][1]==np.inf)):
                length=dgm[i][1][1]-dgm[i][1][0]
                total_0=total_0+length
                l_0.append(length)

    log_l_1=np.log(np.array(l_1)/total_1)
    log_l_0=np.log(np.array(l_0)/total_0)
    prod_1=log_l_1*(np.array(l_1)/total_1)
    prod_0=log_l_0*(np.array(l_0)/total_0)
    S_1=-np.sum(prod_1)
    S_0=-np.sum(prod_0)
    return S_0,S_1

In [2]:
def hyper_vector(time_series,indifference_value,length):
    l=time_series.shape[0]
    N=time_series.shape[1]
    h=np.modf(l/length)[1]
    hyp_ts=[]
    for i in range(1,length+1):
        sub_ts=time_series[:np.int64(h)*i,:]
        hyp=hyper_coherence(sub_ts,indifference_value)
        hyp_ts.append(hyp)
    return hyp_ts

In [1]:
def load_data_mat(path_single_file):
    file_to_open = path_single_file
    data = sio.loadmat(file_to_open)
    key_data = list(data.keys())[-1]
    data = data[key_data]
    return(data)

def load_data_mat_bipolar(path_single_file):
    file_to_open = path_single_file
    data = sio.loadmat(file_to_open)
    key_data = list(data.keys())[-2]
    data = data[key_data]
    return(data)

In [5]:
def logistic(r, x):
    return (1 - r * (x**2))
    # alternative  chaotic map
    # return(4*x*(1-x))


# See book of Kaneko for parameters of eps and/or r to have different regimes
# or https://en.wikipedia.org/wiki/Coupled_map_lattice for notable regimes


# Generate couple map lattice according to this equation: x_i^t= (1-\eps)f[x_i^{t-1}] + \eps/order \sum_{j in \neighbours} f[x_j]^{t-1}
def generate_couple_map(T, N, epsilon, transient_time, r, order=2):
    series = {}

    # Filing the dictionary with N initial random values
    for index_series in range(0, N):
        s = random.random()
        series[index_series] = [s]

    # Generate the coupled maps for a length of size T (yet, we discard the first transient_time elements to remove the transient)
    for i in range(1, T + transient_time + 1):
        for index_series in range(0, N):
            order_k_term = compute_neighbours(N, series, epsilon, index_series, i - 1, r, order)
            new_point = (1 - epsilon) * logistic(r,series[index_series][i - 1]) + order_k_term
            series[index_series].append(new_point)
    return(series)



def compute_neighbours(N, series, epsilon, index_series, i, r, order=2):
    eps_overN = epsilon * (1 / order)
    term_left_right = int(order / 2)

    term = 0
    # Sum over the left neighbors with periodic boundary conditions
    for s in range(1, term_left_right + 1):
        term += logistic(r, series[(index_series - s) % N][i])

    # Sum over the right neighbors with periodic boundary conditions
    for s in range(1, term_left_right + 1):
        term += logistic(r, series[(index_series + s) % N][i])

    # if order is odd, then take the neighbors in an asymmetric way, int(order/2) on the left, int(order/2)+1 on the right
    if order % 2 == 1:
        s = term_left_right + 1
        term += logistic(r, series[(index_series + s) % N][i])

    return(term * eps_overN)